In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box
import cv2
from PIL import Image
from PIL import ImageGrab
import os
from PIL import Image
import pyautogui
import time
from tensorflow.keras import __version__
tf.keras.__version__ = __version__
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

ImportError: cannot import name 'model_from_config' from 'tensorflow.keras.models' (C:\Users\Jordy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\_tf_keras\keras\models\__init__.py)

In [11]:
print(tf.__version__)
print(tf.keras.__version__)

2.16.1
3.0.5


# env

In [62]:
keys = ["z", "s", "q", "d", "e"]

In [63]:
import gym
from gym import spaces


class CustomEnv(Env):
    def __init__(self):
        super(CustomEnv, self).__init__()

        # Define action and observation space
        # They must be gym.spaces objects
        self.total_picked_up = 0
        self.action_space = Discrete(5)
        self.observation_space = Box(low=0, high=255, shape=(1080, 1920, 3), dtype=np.uint8)

    def step(self, action):
        observation = self.get_screen()
        
        pyautogui.press(keys[action])
  
        reward = 0
        if self.item_found(observation):
            reward = 1
            self.total_picked_up += 1
            
        info = {}
        
        done = self.total_picked_up >= 5

        return observation, reward, done, info

    def reset(self):
        self.total_picked_up = 0
        return self.total_picked_up

    def render(self, mode='human'):
        pass
    
    def get_screen(self):
        screen = pyautogui.screenshot()
        return np.array(screen)
    
    def item_found(self, observation):
        
        if len(observation.shape) == 3:
            observation = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Read the template
        template = cv2.imread("resources/template_matching/leafs.png", 0)

        # Perform template matching
        if observation.dtype != np.uint8:
            observation = observation.astype(np.uint8)
        if template.dtype != np.uint8:
            template = template.astype(np.uint8)

        res = cv2.matchTemplate(observation, template, cv2.TM_CCOEFF_NORMED)
        return np.where(res >= .8)

In [64]:
env = CustomEnv()

# model

In [9]:
# # Define loss and optimizer
# optimizer = tf.keras.optimizers.Adam()
# loss_fn = tf.keras.losses.MeanSquaredError()
# 
# # Number of episodes to play
# num_episodes = 500
# 
# 
# # Function to preprocess images
# def preprocess_image(image):
#     return tf.image.rgb_to_grayscale(image)
# 
# 
# for episode in range(num_episodes):
#     initial_state = env.reset()
# 
#     done = False
#     while not done:
#         action = env.action_space.sample()
#         next_state, reward, done, info = env.step(action)
# 
#         # next_state = preprocess_image(next_state)
# 
#         # training here


In [10]:
time.sleep(10)
for episode in range(20):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        # env.render()
        action = env.action_space.sample()  # You can replace with your agent logic
        observation, reward, done, info = env.step(action)
        score += reward
        if score == 5:
            done = True
        print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:0 Score:1
Episode:0 Score:2
Episode:0 Score:3
Episode:0 Score:4
Episode:0 Score:5
Episode:1 Score:1
Episode:1 Score:2
Episode:1 Score:3
Episode:1 Score:4
Episode:1 Score:5
Episode:2 Score:1
Episode:2 Score:2
Episode:2 Score:3
Episode:2 Score:4
Episode:2 Score:5
Episode:3 Score:1
Episode:3 Score:2
Episode:3 Score:3
Episode:3 Score:4
Episode:3 Score:5
Episode:4 Score:1
Episode:4 Score:2
Episode:4 Score:3
Episode:4 Score:4
Episode:4 Score:5
Episode:5 Score:1
Episode:5 Score:2
Episode:5 Score:3
Episode:5 Score:4
Episode:5 Score:5
Episode:6 Score:1
Episode:6 Score:2
Episode:6 Score:3
Episode:6 Score:4
Episode:6 Score:5
Episode:7 Score:1
Episode:7 Score:2
Episode:7 Score:3
Episode:7 Score:4
Episode:7 Score:5
Episode:8 Score:1
Episode:8 Score:2
Episode:8 Score:3
Episode:8 Score:4
Episode:8 Score:5
Episode:9 Score:1
Episode:9 Score:2
Episode:9 Score:3
Episode:9 Score:4
Episode:9 Score:5
Episode:10 Score:1
Episode:10 Score:2
Episode:10 Score:3
Episode:10 Score:4
Episode:10 Score:5
Episo

In [11]:
# while True:
#     # Capture screen
#     screen = pyautogui.screenshot()
#     # Convert the image into numpy array representation
#     frame = np.array(screen)
#     # Convert the BGR image into RGB image
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     # Resize the capturing screen to 1080p
#     frame = cv2.resize(frame, (1920, 1080))
#     # Display screen in 1080p
#     cv2.imshow('Screen Capture in 1080p', frame)
# 
#     # Wait for the user to press the ESC key (ASCII 27) to quit capturing the screen
#     if cv2.waitKey(1) == 27:
#         break
# 
# cv2.destroyAllWindows()

# NN

In [65]:
states = env.observation_space.shape
actions = env.action_space.n

In [66]:
# Define the Convolutional Neural Network
inputs = layers.Input(shape=states)
x = layers.Conv2D(32, (8, 8), strides=4, activation='relu')(inputs)
x = layers.Conv2D(64, (4, 4), strides=2, activation='relu')(x)
x = layers.Conv2D(64, (3, 3), strides=1, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(env.action_space.n, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [67]:
model.build(states)
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 1080, 1920, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 269, 479, 32)   │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 133, 238, 64)   │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 131, 236, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 1978624)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │ 1,013,056,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,013,134,501 (3.77 GB)

 Trainable params: 1,013,134,501 (3.77 GB)

 Non-trainable params: 0 (0.00 B)

# Agent

In [50]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=1000,
                   target_model_update=1e-2)
    return dqn


dqn = build_agent(model, actions)
dqn.compile(keras.optimizers.Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

NameError: name 'BoltzmannQPolicy' is not defined